# Task: Extract and visualize the Seasonal patterns out of the data
## Data: Monthly "page impressions" and "conversion rate" of 100 product categories (Idealo, 100 csv-files)

---

# EDA

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tslearn.clustering import TimeSeriesKMeans, KShape, KernelKMeans
from tslearn.clustering import silhouette_score as ts_silhouette_score
from sklearn.metrics import silhouette_score
import math

### Loading the data

In [2]:
Create a dataframe of all categories and features
*.csv Filenames are: "pis_" + a range of 1 to 100
data = pd.read_csv("./data/pis_1.csv", quoting=2)
data["cat_id"] = 1
data.sort_values(by=["month"], axis=0, ascending=True, inplace=True)

for i in range(2,101):
    add_data = pd.read_csv(f"./data/pis_{i}.csv", quoting=2)
    add_data["cat_id"] = i
    add_data.sort_values(by=["month"], axis=0, ascending=True, inplace=True)
    data = pd.concat([data, add_data], ignore_index=True)
data.columns = ["cat_index", "category", "month", "year", "pageimpressions", "CR", "cat_id"]
data.drop(['cat_index', 'year'], axis=1, inplace=True)
data.head(13)
#data = pd.read_csv("./dash_app/datasets/data.csv", sep=";", index_col=[0])

In [3]:
# impute nan-values with mean of neighnboring values:
data.interpolate(inplace=True)
data.head()

,category,month,pageimpressions,CR,cat_id
0,Herren-Halbschuhe,1.0,86496.0,0.239572,1
1,Herren-Halbschuhe,2.0,75529.0,0.240967,1
2,Herren-Halbschuhe,3.0,84195.0,0.232252,1
3,Herren-Halbschuhe,4.0,92861.0,0.226737,1
4,Herren-Halbschuhe,5.0,93876.0,0.244610,1


Calculate "lead-out" values by multiplying "page impressions" and "conversian rate":

In [4]:
data['lead-out'] = round(data["pageimpressions"] * data["CR"])
data.head()

,category,month,pageimpressions,CR,cat_id,lead-out
0,Herren-Halbschuhe,1.0,86496.0,0.239572,1,20722.0
1,Herren-Halbschuhe,2.0,75529.0,0.240967,1,18200.0
2,Herren-Halbschuhe,3.0,84195.0,0.232252,1,19554.0
3,Herren-Halbschuhe,4.0,92861.0,0.226737,1,21055.0
4,Herren-Halbschuhe,5.0,93876.0,0.244610,1,22963.0


In [5]:
#data.to_csv("./data/data.csv", sep=";")

### Comparing page impressions of some categories

In [6]:
modeleisenbahnen = data.loc[data['category'] == "Modelleisenbahnen"]
fig = px.bar(modeleisenbahnen, x="month", y="pageimpressions", template='plotly_dark',
                  title='Page-impressions in category "Modeleisenbahnen"')
fig.update_layout(xaxis = dict(tickmode = 'linear', dtick = 1))
fig.show()

In [7]:
motorsensen = data.loc[data['category'] == "Motorsensen"]
fig = px.bar(motorsensen, x="month", y="pageimpressions", template='plotly_dark',
                 title='Page-impressions in category "Motorsensen"')
fig.update_layout(xaxis = dict(tickmode = 'linear', dtick = 1))
fig.show()

In [8]:
#gelenkundmuskel = data.loc[data['category'] == "Gelenk- & Muskelpräparate"]
gelenkundmuskel = data.loc[data['cat_id'] == 9]
fig = px.bar(gelenkundmuskel, x="month", y="pageimpressions", template='plotly_dark',
                    title='Page-impressions in category "Gelenk- & Muskelpräparate"')
fig.update_layout(xaxis = dict(tickmode = 'linear', dtick = 1))
fig.show()

Bar Plot with two y-axes for "page impressions" & "conversion rate":

In [9]:
cat_name = "Kinderroller"
cat = data.loc[data['category'] == cat_name]

fig = go.Figure(
    data=[
        go.Bar(name='page impressions', x=cat["month"], y=cat["pageimpressions"], yaxis='y', offsetgroup=1),
        go.Bar(name='conversion rate', x=cat["month"], y=cat["CR"], yaxis='y2', offsetgroup=2)
    ],
    layout={
        'yaxis': {'title': 'page impressions'},
        'yaxis2': {'title': 'conversion rate', 'overlaying': 'y', 'side': 'right'}
    }
)
# Change the bar mode
fig.update_xaxes(title_text="months", dtick=[len(cat.index)])
fig.update_layout(title_text=f"Page Impressions and Conversion Rate of Category: {cat_name}", 
                barmode='group', template='plotly_dark',
                legend=dict(yanchor="bottom",y=0.99, xanchor="right",x=0.99))
fig.show()

Plot and save all categories:

In [10]:
#Plot and save page impressions of all categories:
for cat_id in range(1,101):
    cat = data.loc[data['cat_id'] == cat_id]
    cat_name = cat.category.iloc[0]

    fig = go.Figure(
        data=[
            go.Bar(name='page impressions', x=cat["month"], y=cat["pageimpressions"], yaxis='y', offsetgroup=1),
            go.Bar(name='conversion rate', x=cat["month"], y=cat["CR"], yaxis='y2', offsetgroup=2)
        ],
        layout={
            'yaxis': {'title': 'page impressions'},
            'yaxis2': {'title': 'conversion rate', 'overlaying': 'y', 'side': 'right'}
        }
    )
    # Change the bar mode
    fig.update_xaxes(title_text="months", dtick=[len(cat.index)])
    fig.update_layout(title_text=f"Page Impressions and Conversion Rate of Category: {cat_name}", 
                    barmode='group', template='plotly_dark',
                    legend=dict(yanchor="bottom",y=0.99, xanchor="right",x=0.99))
    #fig.show()
    #fig.write_image(f"./data/plots/cat_PI_{cat_id}.png", format='png', scale=1, width=1200, height=600)

### Page impressions & conversion rate of all categories (monthly mean)

In [11]:
data["lead-out"] = round(data['pageimpressions'] * data['CR'])
data.head()


,category,month,pageimpressions,CR,cat_id,lead-out
0,Herren-Halbschuhe,1.0,86496.0,0.239572,1,20722.0
1,Herren-Halbschuhe,2.0,75529.0,0.240967,1,18200.0
2,Herren-Halbschuhe,3.0,84195.0,0.232252,1,19554.0
3,Herren-Halbschuhe,4.0,92861.0,0.226737,1,21055.0
4,Herren-Halbschuhe,5.0,93876.0,0.244610,1,22963.0


In [12]:
monthly_pi = data['pageimpressions'].groupby(data['month']).mean()
monthly_cr = data['CR'].groupby(data['month']).mean()
monthly_lo = data['lead-out'].groupby(data['month']).mean()

fig = make_subplots(specs=[[{"secondary_y": True, "type": "xy"}]])
fig.add_trace(go.Scatter(y=monthly_pi, x=monthly_pi.index, name="page impressions"), secondary_y=False)
fig.add_trace(go.Scatter(y=monthly_lo, x=monthly_lo.index, name="lead-out"), secondary_y=False)
fig.add_trace(go.Scatter(y=monthly_cr*100, x=monthly_cr.index, name="conversion rate"), secondary_y=True)
fig.update_layout(title_text="Page impressions, conversion rate & lead-out (all categories, mean)", template='plotly_dark')
fig.update_xaxes(title_text="months", dtick=[1,len(monthly_cr.index)])
fig.update_yaxes(title_text="page impressions / lead-out", secondary_y=False, range=[10000,90000])
fig.update_yaxes(title_text="conversion rate %", secondary_y=True, range=[24,26])
fig.show()

# Clustering Seasonality Patterns

https://tslearn.readthedocs.io/en/stable/auto_examples/clustering/plot_kmeans.html#sphx-glr-auto-examples-clustering-plot-kmeans-py

Prepare Data for Clustering:
- page impressions
- conversion rate
- lead-out (pi * cr)

In [13]:
data_pi = data[['category', 'month', 'pageimpressions']]
data_cr = data[['category', 'month', 'CR']]
data_leadout = data[['category', 'month', 'lead-out']]
data_pi.head(3), data_cr.head(3), data_leadout.head(3)


(            category  month  pageimpressions
 0  Herren-Halbschuhe    1.0          86496.0
 1  Herren-Halbschuhe    2.0          75529.0
 2  Herren-Halbschuhe    3.0          84195.0,
             category  month        CR
 0  Herren-Halbschuhe    1.0  0.239572
 1  Herren-Halbschuhe    2.0  0.240967
 2  Herren-Halbschuhe    3.0  0.232252,
             category  month  lead-out
 0  Herren-Halbschuhe    1.0   20722.0
 1  Herren-Halbschuhe    2.0   18200.0
 2  Herren-Halbschuhe    3.0   19554.0)

In [14]:
data_pi_wide =  pd.pivot_table(data_pi, index='month', columns='category', values='pageimpressions')
#data_pi_wide.to_csv("./data/data_pi_wide.csv", sep=";")
data_cr_wide =  pd.pivot_table(data_cr, index='month', columns='category', values='CR')
#data_cr_wide.to_csv("./data/data_cr_wide.csv", sep=";")
data_leadout_wide =  pd.pivot_table(data_leadout, index='month', columns='category', values='lead-out')
#data_leadout_wide.to_csv("./data/data_leadout_wide.csv", sep=";")
data_pi_wide.head()

category,Akkus & Batterien,All in One PCs,"Augen-, Ohren- & Nasenmedikamente",Autobatterien,Bestecke,Betriebssysteme,Bohrer & Bits,Bridgekameras,"Champagner, Sekt & Prosecco",Clogs,...,Trekkingsandalen,Trinkflaschen,USB Sticks,Umhängetaschen,VR-Brillen,Verstärker,WC,Waschtrockner,Xbox One Spiele,Zubehör für Küchengeräte
month,,,,,,,,,,,,,,,,,,,,,
1.0,85837.0,79256.0,66158.0,140228.0,109343.0,73941.0,65603.0,72261.0,59294.0,54682.0,...,25949.0,71637.0,96297.0,48267.0,77844.0,102167.0,72377.0,77931.0,77307.0,81669.0
2.0,57004.0,64025.0,60818.0,81306.0,68012.0,68449.5,52766.0,59766.0,45497.0,47439.0,...,27387.0,34986.0,67531.0,36728.0,40863.0,74299.0,62557.0,52693.0,54913.0,59523.0
3.0,60308.0,69544.0,63310.0,78226.0,71481.0,62958.0,74365.0,76072.0,48162.0,60777.0,...,37763.0,44189.0,83430.0,43493.0,50260.0,80303.0,70651.0,64458.0,66850.0,64530.0
4.0,52951.0,59271.0,61108.0,76142.0,63153.0,48650.0,66463.0,67330.0,47216.0,66324.0,...,54522.0,46096.5,65050.0,44078.0,36173.0,63780.0,54638.0,51924.0,52446.0,55452.0
5.0,53609.0,64724.0,60403.0,69188.0,64833.0,50789.0,67628.0,63150.0,52014.0,63495.0,...,59141.0,48004.0,66729.0,44663.0,39560.0,63375.0,54674.0,54839.0,49444.0,52876.0


In [15]:
# prepare data as 2dim-array for the use in the model:
X = data_pi_wide.transpose().values
X

array([[ 85837. ,  57004. ,  60308. , ...,  76197. ,  90985. ,  93308. ],
       [ 79256. ,  64025. ,  69544. , ...,  88703. , 134463. , 109622. ],
       [ 66158. ,  60818. ,  63310. , ...,  53851. ,  60295. ,  56467. ],
       ...,
       [ 77931. ,  52693. ,  64458. , ...,  72817. , 101172. ,  70408. ],
       [ 77307. ,  54913. ,  66850. , ...,  85936.5,  94533. ,  89577. ],
       [ 81669. ,  59523. ,  64530. , ...,  59478. , 104043. , 118892. ]])

In [16]:
model = TimeSeriesKMeans(n_clusters=6, metric="euclidean", max_iter=100, n_init=2).fit(X)

In [17]:
# Dataframe to map categories to their cluster labels
df_cluster = pd.DataFrame(list(zip(data_pi_wide.columns, model.labels_)), columns=['category', 'cluster'])

# dictionaries and lists for use in plots:
cluster_cat_dict = df_cluster.groupby(['cluster'])['category'].apply(lambda x: [x for x in x]).to_dict()
cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
clusters_all = [cluster for cluster in cluster_len_dict]
clusters_all.sort()
#print(f"df_cluster: {df_cluster}")
#print(f"clusters_all: {clusters_all}")
#print(f"cluster_cat_dict: {cluster_cat_dict}")
#print(f"cluster_len_dict: {cluster_len_dict}")

Make a quality assesment of each cluster with a correlation matrix:

In [18]:
cluster_quality_dict = {}
for cluster_number in clusters_all:
    # get quality score based on the correlation between categories in the cluster
    # For clusters with only one item x_corr_mean is set to 0
    if len(cluster_cat_dict[cluster_number]) > 1:
        x_corr = data_pi_wide[cluster_cat_dict[cluster_number]].corr().abs()
        # get the mean of the values in the upper triangle of the correlation matrix (and round to .2)
        x_corr_mean = round(x_corr.values[np.triu_indices_from(x_corr.values,1)].mean(), 2)
    else:
        x_corr_mean = 0
        # add it to the cluster-quality-dictionary
    cluster_quality_dict[cluster_number] = x_corr_mean
    
correlation_mean = sum(cluster_quality_dict.values())/len(cluster_quality_dict)
cluster_quality_dict, f"Mean cluster quality: {correlation_mean}"

({0: 0, 1: 0.86, 2: 0.84, 3: 0.51, 4: 0.51, 5: 0.83},
 'Mean cluster quality: 0.5916666666666667')

In [19]:
# Dataframe for cluster quality and size:
df_cluster_quality = pd.DataFrame.from_dict(cluster_len_dict, orient='index', columns=['n'])
df_cluster_quality.index.names = ['cluster']
df_cluster_quality['quality_score'] = df_cluster_quality.index.map(cluster_quality_dict)

df_cluster_quality = df_cluster_quality.sort_values('quality_score', ascending=False)
print(f"Mean quality: {df_cluster_quality['quality_score'].mean()}")
print(f"Median quality: {df_cluster_quality['quality_score'].median()}")
df_cluster_quality

Mean quality: 0.5916666666666667
Median quality: 0.6699999999999999


,n,quality_score
cluster,,
1,16,0.86
2,7,0.84
5,8,0.83
4,43,0.51
3,25,0.51
0,1,0.00


Plot each cluster as a line plot (opens in browser):

In [20]:
for cluster_number in clusters_all:
    if len(cluster_cat_dict[cluster_number]) > 1:
        x_corr = data_pi_wide[cluster_cat_dict[cluster_number]].corr().abs()
        # get the mean of the values in the upper triangle of the correlation matrix (and round to .2)
        x_corr_mean = round(x_corr.values[np.triu_indices_from(x_corr.values,1)].mean(), 2)
    else:
        x_corr_mean = 0
    # plot each cluster
    plot_title = f'cluster {cluster_number} (quality={x_corr_mean}, n={cluster_len_dict[cluster_number]})'
    fig = go.Figure()
    cols = cluster_cat_dict[cluster_number]
    ind = data_pi_wide.index
    for i, col in enumerate(cols):
        fig.add_trace(
            go.Scatter(
                x=ind, y=data_pi_wide[col], name=col, line={'width':1}, hoverlabel={'namelength':-1}, showlegend=True # line=dict(width=1) #namelength=-1
            )
        )    
    fig.update_xaxes(title_text="months", dtick=[1,len(data_pi_wide.index)+1])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(title_text=plot_title, template="plotly_dark", height=600)
    fig.show(renderer="browser")

### Normalize Data

In [21]:
def normalize_df(df):
    df_ = df.reset_index()
    df_norm = (df_ - df_.min()) / (df_.max() - df_.min())
    df_norm.drop(['month'], axis=1, inplace=True)
    df_norm = pd.concat((df_norm, df_.month), axis=1)
    df_norm.set_index("month", inplace=True)
    return df_norm

In [22]:
data_pi_wide_norm = normalize_df(data_pi_wide)
data_pi_wide_norm.head(3)

,Akkus & Batterien,All in One PCs,"Augen-, Ohren- & Nasenmedikamente",Autobatterien,Bestecke,Betriebssysteme,Bohrer & Bits,Bridgekameras,"Champagner, Sekt & Prosecco",Clogs,...,Trekkingsandalen,Trinkflaschen,USB Sticks,Umhängetaschen,VR-Brillen,Verstärker,WC,Waschtrockner,Xbox One Spiele,Zubehör für Küchengeräte
month,,,,,,,,,,,,,,,,,,,,,
1.0,0.856407,0.323651,1.000000,1.000000,0.674159,0.698537,0.373614,0.881421,0.412078,0.263929,...,0.087820,0.861849,1.000000,0.209747,0.532727,1.000000,1.000000,0.573693,0.664688,0.516817
2.0,0.302235,0.137054,0.703020,0.282760,0.195287,0.579720,0.000000,0.492641,0.040772,0.000000,...,0.098980,0.000000,0.344036,0.000000,0.059958,0.518430,0.690348,0.110754,0.228778,0.229344
3.0,0.365738,0.204668,0.841611,0.245268,0.235479,0.460903,0.628627,1.000000,0.112493,0.486026,...,0.179505,0.216409,0.706588,0.122969,0.180090,0.622181,0.945574,0.326559,0.461137,0.294339


### Clustering "conversion rate" data the same way

In [23]:
data_cr = data[['category', 'month', 'CR']]
data_cr_wide =  pd.pivot_table(data_cr,index='month',columns='category',values='CR')
#data_cr_wide.to_csv("./data/data_cr_wide.csv", sep=";")


In [24]:
X = data_cr_wide.transpose().values
model_cr = TimeSeriesKMeans(n_clusters=6, metric="euclidean", max_iter=100, n_init=2).fit(X)

In [25]:
# Dataframe to map categories to their cluster labels
df_cluster = pd.DataFrame(list(zip(data_cr_wide.columns, model.labels_)), columns=['category', 'cluster'])

# dictionaries and lists for use in plots:
cluster_cat_dict = df_cluster.groupby(['cluster'])['category'].apply(lambda x: [x for x in x]).to_dict()
cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
clusters_all = [cluster for cluster in cluster_len_dict]
clusters_all.sort()

In [26]:
cluster_quality_dict = {}
for cluster_number in clusters_all:
    if len(cluster_cat_dict[cluster_number]) > 1:
        x_corr = data_cr_wide[cluster_cat_dict[cluster_number]].corr().abs()
        # get the mean of the values in the upper triangle of the correlation matrix (and round to .2)
        x_corr_mean = round(x_corr.values[np.triu_indices_from(x_corr.values,1)].mean(), 2)
    else:
        x_corr_mean = 0
    cluster_quality_dict[cluster_number] = x_corr_mean
cluster_quality_dict

{0: 0, 1: 0.38, 2: 0.46, 3: 0.33, 4: 0.35, 5: 0.48}

In [27]:
cluster_number = 3
df_cluster = pd.DataFrame(list(zip(data_cr_wide.columns, model_cr.labels_)), columns=['category', 'cluster'])

# dictionaries and lists for use in plots:
cluster_cat_dict = df_cluster.groupby(['cluster'])['category'].apply(lambda x: [x for x in x]).to_dict()
cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()

# correlation matrix for each cluster:
x_corr = data_cr_wide[cluster_cat_dict[cluster_number]].corr().abs().values
x_corr_mean = round(x_corr[np.triu_indices(x_corr.shape[0],1)].mean(),2)
# plot each cluster
plot_title = f'Cluster {cluster_number} (quality={x_corr_mean}, n={cluster_len_dict[cluster_number]})'
fig = go.Figure()
cols = cluster_cat_dict[cluster_number]
ind = data_cr_wide.index
for i, col in enumerate(cols):
    fig.add_trace(
        go.Scatter(
            x=ind, y=data_cr_wide[col], name=col, line={'width':1}, hoverlabel={'namelength':-1}, showlegend=True # line=dict(width=1) #namelength=-1
        )
    )    
fig.update_xaxes(title_text="months", dtick=[1,len(data_cr_wide.index)+1])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title_text=plot_title, template="plotly_dark", height=500)
fig.show()

# Quality Assesment of clustering methods

## For a rough quality assesment of a clustering method we can get the mean of the correlation within all clusters:

In [28]:
def correlation_mean(df, labels):
    df_cluster = pd.DataFrame(list(zip(df.columns, labels)), columns=['category', 'cluster'])
    # dictionaries and lists for use in plots:
    cluster_cat_dict = df_cluster.groupby(['cluster'])['category'].apply(lambda x: [x for x in x]).to_dict()
    cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
    clusters_all = [cluster for cluster in cluster_len_dict]
    clusters_all.sort()
    cluster_quality_dict = {}
    for cluster_number in clusters_all:
        # get quality score based on the correlation between categories in the cluster
        # For clusters with only one item x_corr_mean is set to 0
        if len(cluster_cat_dict[cluster_number]) > 1:
            x_corr = df[cluster_cat_dict[cluster_number]].corr().abs()
            # get the mean of the values in the upper triangle of the correlation matrix (and round to .2)
            x_corr_mean = round(x_corr.values[np.triu_indices_from(x_corr.values,1)].mean(), 2)
        else:
            x_corr_mean = 0
            # add it to the cluster-quality-dictionary
        cluster_quality_dict[cluster_number] = x_corr_mean
        
    correlation_mean = sum(cluster_quality_dict.values()) / len(cluster_quality_dict)
    return correlation_mean

In [29]:
def method_quality_corr(df, norm=False, random_state=13, n_init=2, max_iter=25):
    if norm == True:
        x = normalize_df(df).transpose().values
    else:
        x = df.transpose().values
    
    results_df = pd.DataFrame(
            columns=["n_clusters", "KMeans euclidean", "KMeans dtw", "KMeans soft dtw", "KernelMeans", "KShape"])

    for n_clusters in range(2,13):
        kmeans_euc_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_euc_labels = kmeans_euc_model.labels_
        kmeans_euc_corr_mean = correlation_mean(df, kmeans_euc_labels)

        kmeans_dtw_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_dtw_labels = kmeans_dtw_model.labels_
        kmeans_dtw_corr_mean = correlation_mean(df, kmeans_dtw_labels)

        kmeans_sdtw_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="softdtw", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_sdtw_labels = kmeans_sdtw_model.labels_
        kmeans_sdtw_corr_mean = correlation_mean(df, kmeans_sdtw_labels)

        kernel_means_model = KernelKMeans(n_clusters=n_clusters, kernel="gak", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kernel_means_labels = kernel_means_model.labels_
        kernel_means_corr_mean = correlation_mean(df, kernel_means_labels)

        kshape_model = KShape(n_clusters=n_clusters, max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kshape_labels = kshape_model.labels_
        kshape_corr_mean = correlation_mean(df, kshape_labels)

        results = {
            "n_clusters": n_clusters, 
            "KMeans euclidean": kmeans_euc_corr_mean, 
            "KMeans dtw": kmeans_dtw_corr_mean, 
            "KMeans soft dtw": kmeans_sdtw_corr_mean, 
            "KernelMeans": kernel_means_corr_mean, 
            "KShape": kshape_corr_mean
        }
        results_df = pd.concat([results_df, pd.DataFrame.from_records([results])], ignore_index=True)

    results_df.set_index('n_clusters', inplace=True)    
    return results_df

Calculate Clusters and quality assessment for 2 to 12 clusters. The results are saved to csv-files for later use.

In [30]:
# not normalized data:
data_pi_wide = pd.read_csv("./dash_app/datasets/data_pi_wide.csv", sep=";", index_col="month")

method_quality_df = method_quality_corr(
    data_pi_wide, 
    norm=False,
    random_state=15,
    n_init=6,
    max_iter=100,
)
method_quality_df.to_csv("./dash_app/datasets/method_quality_corr.csv", sep=";")

In [31]:
# normalized data:
method_quality_df = method_quality_corr(
   data_pi_wide, 
   norm=True,
   random_state=16,
   n_init=8,
   max_iter=100,
)
method_quality_df.to_csv("./dash_app/datasets/method_quality_corr_norm.csv", sep=";")

A plot of the results for not normalized data:

In [32]:
df = pd.read_csv("./dash_app/datasets/method_quality_corr.csv", sep=";", index_col="n_clusters")

fig = go.Figure()
plot_title = "Clustering quality measurement of all methods with correlation mean (not normalized data)"
# Loop dataframe columns and plot columns to the figure
for i in range(0, len(df.columns)):
    col_name = df.columns.values[i]
    fig.add_trace(go.Scatter(x=df.index, y=df[col_name], mode='lines', name=col_name))
fig.update_xaxes(title_text="n_clusters", dtick=[2,len(df.index)+1])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title_text=plot_title, template="plotly_dark", height=500)
fig.show()

A plot of the results for not normalized data:

In [33]:
df = pd.read_csv("./dash_app/datasets/method_quality_corr_norm.csv", sep=";", index_col="n_clusters")

fig = go.Figure()
plot_title = "Clustering quality measurement of all methods with correlation mean (normalized data)"
# Loop dataframe columns and plot columns to the figure
for i in range(0, len(df.columns)):
    col_name = df.columns.values[i]
    fig.add_trace(go.Scatter(x=df.index, y=df[col_name], mode='lines', name=col_name))#, line_shape="spline"))
fig.update_xaxes(title_text="n_clusters", dtick=[2,len(df.index)+1])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title_text=plot_title, template="plotly_dark", height=500)
fig.show()

## Quality Assesment of clustering methods with "silhouette score"

https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient

If the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient (sklearn.metrics.silhouette_score) is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The Silhouette Coefficient is defined for each sample and is composed of two scores:

    a: The mean distance between a sample and all other points in the same class.
    b: The mean distance between a sample and all other points in the next nearest cluster.

The Silhouette Coefficient s for a single sample is then given as:

$$s = \frac{b - a}{max(a, b)}$$



The Silhouette Coefficient for a set of samples is given as the mean of the Silhouette Coefficient for each sample.

In [34]:
def method_quality(df, norm=False, random_state=13, n_init=2, max_iter=25):
    if norm == True:
        x = normalize_df(df).transpose().values
    else:
        x = df.transpose().values
    
    results_df = pd.DataFrame(
            columns=["n_clusters", "KMeans euclidean", "KMeans dtw", "KMeans soft dtw", "KernelMeans", "KShape"])

    for n_clusters in range(2,13):
        kmeans_euc_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_euc_labels = kmeans_euc_model.labels_
        kmeans_euc_score = silhouette_score(x, kmeans_euc_labels)

        kmeans_dtw_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_dtw_labels = kmeans_dtw_model.labels_
        kmeans_dtw_score = silhouette_score(x, kmeans_dtw_labels)

        kmeans_sdtw_model = TimeSeriesKMeans(n_clusters=n_clusters, metric="softdtw", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kmeans_sdtw_labels = kmeans_sdtw_model.labels_
        kmeans_sdtw_score = silhouette_score(x, kmeans_sdtw_labels)

        kernel_means_model = KernelKMeans(n_clusters=n_clusters, kernel="gak", max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kernel_means_labels = kernel_means_model.labels_
        kernel_means_score = silhouette_score(x, kernel_means_labels)

        kshape_model = KShape(n_clusters=n_clusters, max_iter=max_iter, n_init=n_init, random_state=random_state).fit(x)
        kshape_labels = kshape_model.labels_
        kshape_score = silhouette_score(x, kshape_labels)

        results = {
            "n_clusters": n_clusters, 
            "KMeans euclidean": kmeans_euc_score, 
            "KMeans dtw": kmeans_dtw_score, 
            "KMeans soft dtw": kmeans_sdtw_score, 
            "KernelMeans": kernel_means_score, 
            "KShape": kshape_score
        }
        results_df = pd.concat([results_df, pd.DataFrame.from_records([results])], ignore_index=True)

    results_df.set_index('n_clusters', inplace=True)    
    return results_df

In [35]:
# not normalized data:
data_pi_wide = pd.read_csv("./dash_app/datasets/data_pi_wide.csv", sep=";", index_col="month")

method_quality_df = method_quality(
    data_pi_wide, 
    norm=False,
    random_state=15,
    n_init=6,
    max_iter=100,
)
method_quality_df.to_csv("./dash_app/datasets/method_quality.csv", sep=";")

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site

In [36]:
# normalized data:
method_quality_df = method_quality(
   data_pi_wide, 
   norm=True,
   random_state=16,
   n_init=8,
   max_iter=100,
)
method_quality_df.to_csv("./dash_app/datasets/method_quality_norm.csv", sep=";")

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning:

2-Dimensional data passed. Assuming these are 100 1-dimensional timeseries

c:\Users\torst\miniconda3\lib\site

Plot the results for not normalized data:

In [37]:
df = pd.read_csv("./dash_app/datasets/method_quality.csv", sep=";", index_col="n_clusters")

fig = go.Figure()
plot_title = "Clustering quality measurement of all methods with silhouette score (not normalized data)"
# Loop dataframe columns and plot columns to the figure
for i in range(0, len(df.columns)):
    col_name = df.columns.values[i]
    fig.add_trace(go.Scatter(x=df.index, y=df[col_name], mode='lines', name=col_name))
fig.update_xaxes(title_text="n_clusters", dtick=[2,len(df.index)+1])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title_text=plot_title, template="plotly_dark", height=500)
fig.show()

Results for normalized data:

In [38]:
df = pd.read_csv("./dash_app/datasets/method_quality_norm.csv", sep=";", index_col="n_clusters")

fig = go.Figure()
plot_title = "Clustering quality measurement of all methods with silhouette score (normalized data)"
# Loop dataframe columns and plot columns to the figure
for i in range(0, len(df.columns)):
    col_name = df.columns.values[i]
    fig.add_trace(go.Scatter(x=df.index, y=df[col_name], mode='lines', name=col_name))#, line_shape="spline"))
fig.update_xaxes(title_text="n_clusters", dtick=[2,len(df.index)+1])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(title_text=plot_title, template="plotly_dark", height=500)
fig.show()

### Some results of the quality assessment:

- KMeans euclidean is very fast and performs well
- KernelMeans performs well only on the normalized data
- Soft dtw is slow on the unnormalized data
- KShape behaves pretty randomly
- Silhouette score decreases with n_clusters increasing
- The correlation mean score increases with n_clusters on normalized data

### Some results of the seasonality-clustering:

There are several typical clusters of page impressions:
  - relatively low throughout the year and high before christmas (Spielzeug-Fahrzeuge, Kuscheltiere)
  - high in winter (Skihelme & Snowboardhelme, Thermostate)
  - high in summer (Gartenschläuche, Motorsensen, Sonnenpflege, Sonnenschirme)
  - high before summer (Gartenstühle, Fahrradschlösser, Dünger)
  - high around easter and before christmas (Kinderroller)
  - no suitable clusters (Autobatterien, Steuersoftware)